In [1]:
import numpy as np
import plotly.graph_objects as go
from collections import deque

# -------------------------------
# 1. Minkowski bilinear form
# -------------------------------
def minkowski_dot(x, y):
    """
    Bilinear form B(x,y) = -x0*y0 + x1*y1 + x2*y2 + x3*y3
    Both x,y are length-4 NumPy arrays.
    """
    return -x[0]*y[0] + x[1]*y[1] + x[2]*y[2] + x[3]*y[3]

# -------------------------------
# 2. Build the Gram matrix for (4,3,5)
#    Coxeter diagram: a chain of 4 nodes:
#       --4--  --3--  --5--
#    Angles: pi/4, pi/3, pi/5 along edges. The others are pi/2 (orthogonal).
# -------------------------------
def coxeter_435_gram_matrix():
    """
    Return the 4x4 Gram matrix G where G[i,i] = B(vi, vi),
    G[i,j] = B(vi, vj) for i!=j. We want B(vi, vi) > 0 (spacelike).
    
    The reflection planes satisfy:
      G[i,i] = 1
      G[i,j] = -cos(pi/m_ij) if there's an edge label m_ij,
               0 (=> angle = pi/2 => cos(pi/2)=0) if no edge label.
    For the chain (4,3,5) we have edges:

       v0 --4-- v1 --3-- v2 --5-- v3

    => G[0,1] = G[1,0] = -cos(pi/4) = -sqrt(2)/2
    => G[1,2] = G[2,1] = -cos(pi/3) = -0.5
    => G[2,3] = G[3,2] = -cos(pi/5) = -(1+sqrt{5})/4 ~ -0.80901696
    => The others = 0
    """
    G = np.eye(4)
    G[0,1] = G[1,0] = -np.cos(np.pi/4)  # -sqrt(2)/2
    G[1,2] = G[2,1] = -np.cos(np.pi/3)  # -0.5
    G[2,3] = G[3,2] = -np.cos(np.pi/5)  # ~ -0.809016964
    return G

# -------------------------------
# 3. Extract vectors v0, v1, v2, v3 in Minkowski space
#    from the Gram matrix.
# -------------------------------
def find_reflection_vectors(G):
    """
    Given the 4x4 Gram matrix G = [B(v_i, v_j)],
    we want to find an actual set of vectors v0,...,v3 in R^{3,1}
    that realize those dot products. We do a "pseudo-Cholesky"
    style factorization. There's more than one solution, but
    we just need one consistent set.
    
    This routine attempts a simple linear-algebra factorization
    in signature (3,1). We'll do a standard Cholesky in +--- or -+++,
    but we must be careful with sign.
    
    We'll do a naive approach:
      1) diagonalize G in a standard Euclidean sense
         or do an LDL^T factoring,
      2) pick out a basis that yields approximate Minkowski vectors.
    
    For a small example, we can also attempt to solve incrementally.
    For robust usage, a dedicated indefinite factorization is better.
    """
    # We'll do an eigendecomposition in the usual sense, even though
    # it's indefinite. Then we try to reconstruct approximate vectors
    # from the principal component.  This is not the only approach!
    w, V = np.linalg.eig(G)
    
    # w = eigenvalues, V = eigenvectors (columns).
    # Because G is indefinite but rank=4, we expect positive and negative eigenvalues.
    # We'll reorder them by absolute value descending, just to be safe.
    idx_sorted = np.argsort(-np.abs(w))
    w = w[idx_sorted]
    V = V[:, idx_sorted]
    
    # Now, G = V diag(w) V^T in Euclidean sense. We want a matrix M s.t. M^T * M ~ G in Minkowski sense.
    # A simpler approach for a small dimension: we can do a "square root" of diag(w), taking sign into account.
    # Then some linear combination. We'll attempt a simpler direct method:
    
    # We'll do: let L = V * sqrt(diag(w)) in the real sense (with sign for negative w).
    # Then the columns of L^T might be an approximation to v_i. We then might need a local fix
    # to ensure B(v_i, v_j) matches G. There's a bit of nuance, but let's attempt a direct approach:
    
    # Build sqrt of diag(w) with sign
    w_sqrt = np.zeros_like(w)
    for i in range(len(w)):
        if w[i] > 0:
            w_sqrt[i] = np.sqrt(w[i])
        else:
            w_sqrt[i] = np.sqrt(-w[i]) * 1j  # purely imaginary for negative w
    
    # Construct L in complex sense
    L_complex = V @ np.diag(w_sqrt)
    # The vectors we want might come from columns of L_complex^T, but we only want real Minkowski vectors.
    # In indefinite forms, it is typical that some directions become imaginary. We'll do a simpler approach:
    # Instead, let's do a smaller direct solve. We'll attempt to solve for v_i in R^4 incrementally:
    
    # We'll do a simpler "incremental" method for 4 vectors:
    #   We want B(v0, v0) = G[0,0], B(v1, v1) = G[1,1], etc., and B(v0,v1)=G[0,1], etc.
    
    # Because we know G[i,i]=1, let's just pick an explicit coordinate system:
    # Start v0, v1, v2, v3 in R^4, solve step by step. For dimension 4, we can do it.
    
    v = np.zeros((4,4))  # v[i] is the Minkowski vector for reflection i.
    
    # We'll place v0 along the "x1" axis in Minkowski space for simplicity (with B(v0,v0)=1).
    # In Minkowski form, B(e1, e1)=+1, B(e2,e2)=+1, B(e3,e3)=+1, B(e0,e0)=-1.
    # Let's do v0 = (0,1,0,0). Then B(v0,v0)=1. Great.
    v[0] = np.array([0,1,0,0], dtype=float)  # v0
    
    # Next, solve for v1 s.t. B(v1,v1)=1 and B(v0,v1) = G[0,1].
    # Let v1 = (x0, x1, x2, x3). Then B(v0,v1)= -0*x0 +1*x1 +0*x2 +0*x3 = x1.
    # We want x1 = G[0,1] = -cos(pi/4).
    # Also B(v1,v1)= -x0^2 + x1^2 + x2^2 + x3^2 = 1.
    # For simplicity, set x0=x2=x3=0 => x1^2=1 => x1= +1 or -1. But we need x1= G[0,1].
    # Actually that won't match unless G[0,1] is ±1. But G[0,1] ~ -0.707..., so we need a different approach.
    #
    # We'll choose x1 = G[0,1]. Then we solve -x0^2 + x1^2 = 1 => -x0^2 + (-0.7071)^2=1 => x0^2= x1^2 -1
    # That might be negative. Indeed, x1^2 ~ 0.5, so x1^2 -1= -0.5 => x0^2=0.5 => x0= sqrt(0.5).
    # We'll pick x2=0, x3=0 for simplicity, and choose sign so that B(...)>0 in the "time" direction if needed.
    
    v1 = np.zeros(4)
    x1 = G[0,1]  # -0.707106...
    # Solve -x0^2 + x1^2 = 1 => x0^2 = x1^2 -1 = 0.5 -1= -0.5 => x0= sqrt(0.5)
    x0 = np.sqrt(0.5)  # = ~0.707106...
    # We have to be mindful of sign for Minkowski. B(v1,v1) = -x0^2 + x1^2 => -0.5 + 0.5= 0 => 0 not 1! Let's do it carefully:
    # Actually x1^2=0.5 => x1^2 -1= -0.5 => that means x0^2=0.5 => -x0^2 + x1^2= -0.5 +0.5=0, not 1. We need 1, so we need another component:
    # We'll add x2 as well. Let x2^2 = alpha. Then we want:
    #   -x0^2 + x1^2 + x2^2 = 1
    # We have x1^2=0.5, we want x1= -0.707..., so x1^2=0.5
    # So we choose x0^2 + x2^2= 1. Let's pick x0=0, x2=1. Then B(v1,v1)=0 +0.5 +1=1.5 not 1. So let's do x1^2=0.5 => we can't fix x1 that easily.
    #
    # Instead of hand-solving each step, let's do a small system solve approach for v1, v2, v3 that meets the pairwise dot constraints with v0, etc.  That’s simpler:

    # We want:
    #   B(v0,v0)=1, B(v1,v1)=1, B(v2,v2)=1, B(v3,v3)=1
    #   B(v0,v1) = G[0,1], B(v1,v2) = G[1,2], B(v2,v3)=G[2,3], ...
    #   B(v0,v2)=0 (since G[0,2]=0), etc.
    #
    # Let's do a small numerical solve with unknowns x_{i,0..3} for v_i in R^4. We'll treat them as 16 unknowns.

    # We'll store them in a 16D vector: v_0 = (x00,..,x03), v_1= (x10,..), ...
    # Then we impose constraints from G. We'll use a standard nonlinear solver, e.g. scipy.optimize.

    import scipy.optimize as opt

    # Flatten v into a length-16 vector
    def pack_vs(vmat):
        return vmat.flatten()

    def unpack_vs(x):
        return x.reshape((4,4))

    def constraints(x):
        """ Return array of constraints from G[i,j] = B(vi, vj). """
        vmat = unpack_vs(x)
        eqs = []
        for i in range(4):
            # B(v_i, v_i) = 1
            eqs.append( minkowski_dot(vmat[i], vmat[i]) - G[i,i] )
            for j in range(i+1,4):
                eqs.append( minkowski_dot(vmat[i], vmat[j]) - G[i,j] )
        return np.array(eqs)

    # initial guess: small random vectors, hopefully solver converges
    rng = np.random.default_rng(42)
    x0 = rng.normal(scale=0.1, size=16)
    # we can attempt a structured guess: e.g. v0 along e1, v1 in span(e0,e1,e2)...

    sol = opt.root(constraints, x0, method='hybr')
    if not sol.success:
        raise ValueError("Could not solve for reflection vectors from Gram matrix. Try another init.")

    vmat = unpack_vs(sol.x)
    return vmat

# -------------------------------
# 4. Reflection in plane normal to v_i
# -------------------------------
def reflect(x, v):
    """
    Reflect point x in Minkowski space across plane orthonormal to v.
    R_v(x) = x - 2 * B(x,v)/B(v,v) * v
    """
    denom = minkowski_dot(v,v)
    num = 2.0 * minkowski_dot(x,v)
    return x - (num/denom)*v

# -------------------------------
# 5. Poincaré ball projection
#    If p in H^3 ( -B(p,p)=1, p0>0 ), then
#      Poincare(p) = p_{1..3} / (1 + p0).
# -------------------------------
def poincare_projection(p):
    denom = 1.0 + p[0]  # p0 is the "time" component in our signature
    return np.array([p[1]/denom, p[2]/denom, p[3]/denom])

# Hyperbolic distance function, for BFS cutoff
def hyperbolic_distance(p, q):
    """
    d(p,q) = arcosh( -B(p,q) ).
    p,q in H^3 => B(p,p) = B(q,q) = -1, p0>0, q0>0.
    """
    return np.arcosh( -minkowski_dot(p,q) )

In [2]:
# -------------------------------
# MAIN CODE
# -------------------------------
def generate_435_honeycomb(radius_cutoff=2.0, max_cells=5000):
    """
    Generate the (4,3,5) honeycomb up to hyperbolic distance 'radius_cutoff'
    from the "center" point center_pt = (1,0,0,0) in Minkowski coords
    (which maps to origin in Poincaré ball).
    
    Returns a list of vertex sets in Poincaré coords. Each set corresponds
    to an image of the fundamental tetrahedron (or just its vertices).
    
    By default we only store the tetrahedron's 4 vertices under group action,
    but you could store edges or full faces if you want more detail.
    """
    # 1) Build Gram matrix, solve for reflection vectors
    G = coxeter_435_gram_matrix()
    reflection_vecs = find_reflection_vectors(G)  # shape (4,4): v0,v1,v2,v3
    
    # 2) Fundamental tetrahedron vertices
    #    Easiest approach: the fundamental domain is the intersection of
    #    half-spaces B(x, v_i) <= 0 for i=0..3.  We can find the 4-plane intersection.
    #    Or simpler: take each triple of planes v_i, v_j, v_k and solve B(x, v_i)=0, etc.
    #    We can do that for the 4 choose 3=4 corners.
    
    def plane_intersection(v1, v2, v3):
        """
        Solve for x in Minkowski space s.t. B(x,v1)=0, B(x,v2)=0, B(x,v3)=0,
        and B(x,x)=-1, x0>0. We'll do a simple linear solve for the subspace,
        then scale to get -B(x,x)=1, ensure x0>0.
        """
        # Solve linear system A*x = 0 with A = [v1, v2, v3]^T in R^4
        A = np.array([v1, v2, v3])  # shape (3,4)
        # We want A dot x = 0 => A * x = 0, in usual Eucl. sense for row * x,
        # but we want B(x,vi)=0 => minkowski_dot(x, vi) = 0 => that's a different expression.
        # However, if we treat B(x, vi)= x^T * M * vi = 0, with M= diag(-1,1,1,1).
        # It's simpler to do direct approach with a small dimension:
        
        # We'll param by x( t ) and solve constraints. We'll just do an SVD approach:
        # We want x s.t. minkowski_dot(x, v1)=0 => -x0 v1[0] + x1 v1[1] ...
        # We'll do a small solver with e.g. least_squares plus the constraint B(x,x)=-1 afterwards.
        
        # For simplicity, let's do a naive approach: we want x in the orth. complement in Minkowski sense
        # of v1,v2,v3. That means x is in the span of w such that B(w, v_i)=0 for i=1..3.
        # We'll take the cross-product approach generalized to Minkowski? It's easier to do iterative solves:
        
        # We'll do a simple numeric approach again:
        def f_x(vars):
            # x = (vars[0], vars[1], vars[2], vars[3])
            x_ = vars
            return [
                minkowski_dot(x_, v1),
                minkowski_dot(x_, v2),
                minkowski_dot(x_, v3),
            ]
        
        x0_guess = np.random.normal(size=4)
        sol_local = opt.root(f_x, x0_guess, method='hybr')
        if not sol_local.success:
            return None
        
        x_ = sol_local.x
        # Now scale x_ so that B(x_, x_) = -1
        norm_ = minkowski_dot(x_, x_)
        if norm_ > 0: 
            # The intersection might be outside H^3 or purely imaginary solution.
            # We'll pick x_ -> x_ * i for Minkowski? Instead let's forcibly set -norm_>0 => norm_<0 needed
            # We can check if norm_ < 0, then scale factor to get -1. If norm_>0, we can multiply by 1i
            # but that won't be real. Sometimes solutions won't exist if the planes don't meet in H^3.
            return None
        
        scale = np.sqrt((-1.0)/norm_)  # we want -minkowski_dot(scale*x_, scale*x_)=1 => scale^2 * norm_ = -1 => scale= sqrt(-1/norm_)
        x_ = x_ * scale
        
        # ensure x0>0
        if x_[0] <= 0:
            x_ = -x_
        return x_
    
    tet_verts = []
    # corners from choosing triple of the 4 reflection planes
    plane_indices = [(0,1,2), (0,1,3), (0,2,3), (1,2,3)]
    for trip in plane_indices:
        vs = reflection_vecs[list(trip)]
        x_ = plane_intersection(vs[0], vs[1], vs[2])
        if x_ is not None:
            tet_verts.append(x_)
    tet_verts = np.array(tet_verts)  # shape (4,4)

    # We store as the fundamental "cell" the 4 vertices. If you want edges or faces,
    # you'd also gather the line segments or polygons.

    # 3) BFS over group reflections
    center_pt = np.array([1.,0.,0.,0.])  # Minkowski "center" (maps to origin in Poincaré)
    
    # We'll store a queue of group elements in terms of the composition of reflections.
    # But we only need to store the Minkowski transform of the tetrahedron's vertices,
    # and also keep track of the "distance from center".
    
    # We'll represent a state as (verts, dist). We'll expand by applying each reflection R_i.
    
    seen = set()
    queue = deque()
    
    # Convert the fundamental tetrahedron's vertices to a canonical "tuple" so we can store them in sets
    # But it’s large to store all 4 vertices. Instead, store a group element signature.
    # A simpler approach is to store just the average of the cell's vertices as a "representative point."
    # Then we can track BFS from there. Let's define that:
    center_of_fund = np.mean(tet_verts, axis=0)
    # project center_of_fund onto H^3 (just to ensure B(...)=-1) if needed:
    # (In principle the center might not lie exactly on H^3 if the domain is not symmetrical around center.)
    # We'll skip that or do a quick fix:
    nm = minkowski_dot(center_of_fund, center_of_fund)
    if nm < 0:
        sc = np.sqrt((-1.0)/nm)
        center_of_fund *= sc
    
    dist_0 = hyperbolic_distance(center_of_fund, center_pt)
    queue.append((center_of_fund, tet_verts))
    seen.add(tuple(np.round(center_of_fund,5)))  # approximate
    
    # Data for plotting
    all_cells = []  # list of arrays, each shape (4,4) Minkowski coords of tetrahedron
    
    while queue and len(all_cells) < max_cells:
        cell_center, cell_verts = queue.popleft()
        # Check distance
        dC = hyperbolic_distance(cell_center, center_pt)
        if dC <= radius_cutoff:
            all_cells.append(cell_verts)
            # Expand neighbors by reflecting in each generating plane
            for i in range(4):
                # Reflect center, reflect each vertex
                new_center = reflect(cell_center, reflection_vecs[i])
                new_verts  = np.array([reflect(vx, reflection_vecs[i]) for vx in cell_verts])
                # Check if we've seen
                rep = tuple(np.round(new_center,5))
                if rep not in seen:
                    seen.add(rep)
                    queue.append((new_center, new_verts))
    
    # 4) Now we have a bunch of tetrahedra in Minkowski coords. Let's project to Poincaré and return.
    all_cells_poincare = []
    for cell_verts_mink in all_cells:
        cell_3d = np.array([poincare_projection(v) for v in cell_verts_mink])
        all_cells_poincare.append(cell_3d)

    return all_cells_poincare


# -------------------------------
# PLOTLY VISUALIZATION
# -------------------------------
def plot_honeycomb_poincare(all_cells_poincare):
    """
    Takes a list of tetrahedron vertex-sets (in R^3) and creates a Plotly 3D figure
    showing them. We'll just show the edges of each tetrahedron for clarity.
    """
    fig = go.Figure()
    
    # Plot each tetrahedron as edges
    # Each tetrahedron has 6 edges between its 4 vertices.
    edges = [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]
    
    for cell in all_cells_poincare:
        for (i,j) in edges:
            x_vals = [cell[i,0], cell[j,0]]
            y_vals = [cell[i,1], cell[j,1]]
            z_vals = [cell[i,2], cell[j,2]]
            fig.add_trace(go.Scatter3d(
                x=x_vals, 
                y=y_vals, 
                z=z_vals,
                mode='lines',
                line=dict(color='blue', width=2),
                hoverinfo='none'
            ))
    # Make the Poincaré ball boundary (the unit sphere in R^3) for reference
    # param for a sphere
    sphere_u = np.linspace(0, np.pi, 24)
    sphere_v = np.linspace(0, 2*np.pi, 24)
    xs = []
    ys = []
    zs = []
    for u in sphere_u:
        row_x = []
        row_y = []
        row_z = []
        for v in sphere_v:
            x_ = np.sin(u)*np.cos(v)
            y_ = np.sin(u)*np.sin(v)
            z_ = np.cos(u)
            row_x.append(x_)
            row_y.append(y_)
            row_z.append(z_)
        xs.append(row_x)
        ys.append(row_y)
        zs.append(row_z)
    fig.add_trace(go.Surface(
        x=xs, 
        y=ys, 
        z=zs,
        opacity=0.1, 
        colorscale='Gray', 
        showscale=False,
        name='Poincaré ball'
    ))

    fig.update_layout(
        scene=dict(
            aspectmode='cube',
            xaxis=dict(range=[-1,1], showgrid=False, showticklabels=False, zeroline=False),
            yaxis=dict(range=[-1,1], showgrid=False, showticklabels=False, zeroline=False),
            zaxis=dict(range=[-1,1], showgrid=False, showticklabels=False, zeroline=False),
        ),
        title='(4,3,5) Honeycomb in Poincaré Ball Model (Edges of Some Fundamental Copies)'
    )
    
    return fig

# -------------------------------
# Putting it all together in a single cell
# -------------------------------
if __name__ == "__main__":
    # Parameters
    RADIUS_CUTOFF = 2.0  # hyperbolic distance cutoff
    MAX_CELLS     = 100  # BFS limit
    
    cells_3d = generate_435_honeycomb(radius_cutoff=RADIUS_CUTOFF, max_cells=MAX_CELLS)
    fig = plot_honeycomb_poincare(cells_3d)
    fig.show()

/var/folders/wn/2c3pn3dj1bq4tv3jm7kbrr0w0000gn/T/ipykernel_65603/1896928477.py:95: ComplexWarning: Casting complex values to real discards the imaginary part
  w_sqrt[i] = np.sqrt(-w[i]) * 1j  # purely imaginary for negative w


TypeError: fsolve: there is a mismatch between the input and output shape of the 'func' argument 'constraints'.Shape should be (16,) but it is (10,).